<a href="https://colab.research.google.com/github/MxD-lab/SNN_Simulation/blob/neralnetwork/neralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
import numpy 



In [2]:
#シグモイド関数（出力層）
def sigmoid(x):
  return 1/(1+np.exp(-x))

#ReLU関数（隠れ層）
def relu(x):
  return np.maximum(0,x)

In [3]:
#交差エントロピー誤差

def cross_entropy_error(y,t):
  delta:float
  delta = 1e-7
  return -np.sum(t*np.log(y+delta))

In [4]:
#順方向処理
x = np.array([[0,0],[1,1],[0,1],[1,0]])#入力
W1 = np.array([[0.5,0.5,0.5],[0.5,0.5,0.5]])#1層目の重み
A1 = np.dot(x,W1)
Z1 = relu(A1)#一層目活性化関数かける

W2 = np.array([[0.5],[0.5],[0.5]])
A2 = np.dot(Z1,W2)
Z2 = sigmoid(A2)

Z2

array([[0.5       ],
       [0.81757448],
       [0.6791787 ],
       [0.6791787 ]])

In [5]:
#活性化関数の微分(relu)u:隠れ層の活性
def relu_diff(u):
  d = np.zeros_like(u) #xと同じ形状の配列を作成
  for i in range(len(u)):
    for j in range(len(u[0])):
      if u[i][j]>0:
        d[i] = 1
      else: 
        d[i][j] = 0

  return d

In [6]:
#隠れ層の誤差
def hidden_delta(delta,w,activ):
  d = np.zeros((4,3)) #データ数×隠れ層のニューロン数の配列を作成
  for i in range(len(delta)):
    for j in range(len(w[0])):
      d[i][j] = delta[i]*w[j]*activ[i][j]
  return np.array(d)
  

In [7]:
"""
逆方向処理
target:教師データ
learning_rate:学習率
L：損失関数
"""
N = 4#データ数
target =  np.array([[0],[0],[1],[1]]) #教師
learning_rate = 0.5#学習率

#print(W2)
###出力層の逆伝搬
#L = cross_entropy_error(Z2,target)#出力層の損失関数
delta = Z2 - target#誤差
#print(delta)
Z1_t = np.transpose(Z1)
sum_delta = np.dot(Z1_t,delta)
delta_out = 1/N*sum_delta
W2 = W2 - learning_rate*delta_out
#print(W2)

###隠れ層の逆伝搬

differential_y = relu_diff(A1)#活性化関数の微分
print(differential_y)
hid_delta = hidden_delta(delta,W2,differential_y)
x_t = np.transpose(x)
delta_hidden = np.dot(x_t,hid_delta)
W1 = W1 - learning_rate*delta_hidden
W1


[[0. 0. 0.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


array([[0.39123444, 0.5       , 0.5       ],
       [0.39123444, 0.5       , 0.5       ]])